In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf

tf.get_logger().setLevel("ERROR")
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp


# Using Bicubic Downscaling


In [ ]:
scale_factor = 4
resize_method = "bicubic"
builder_kwargs = {
    "resize_method": resize_method,
    "scale": scale_factor,
    "antialias": True,
}

dataset = tfds.load("vid4", split="test", builder_kwargs=builder_kwargs).map(
    datasets.compose(
        datasets.map_on_dict(datasets.to_float32),
        datasets.map_on_dict(datasets.from_255_to_1_range),
    )
)

example = datasets.get_one_example(dataset, index=0)
video_hr = example["hr"]
video_lr = example["lr"]
ref_index = tf.shape(video_hr)[0] // 2


### Reconstruct


In [ ]:
# Load the denoiser
denoiser, (denoiser_min, denoiser_max) = dppp.load_denoiser("models/drugan+_0.0-0.2.h5")
num_frames = 9
resize_fn = dppp.create_resize_fn(resize_method, True)

# Crop the video
video_frames = video_lr.shape[0]
if video_frames > num_frames:
    start_index = max(0, ref_index - (num_frames // 2))
    end_index = start_index + num_frames
    video_lr_cropped = video_lr[start_index:end_index]
    ref_index_cropped = ref_index - start_index
else:
    video_lr_cropped = video_lr
    ref_index_cropped = ref_index

# Callbacks: Print the PSNR every 5th step
callbacks = [dppp.callback_print_psnr("psnr", 5, video_hr[ref_index])]

reconstructed = dppp.hqs_video_super_resolve(
    degraded=video_lr_cropped,
    ref_index=ref_index_cropped,
    sr_factor=scale_factor,
    denoiser=denoiser,
    max_denoiser_stddev=denoiser_max,
    resize_fn=resize_fn,
    callbacks=callbacks,
)


In [ ]:
# Print PSNR and LPIPS
psnr = dppp.psnr(video_hr[ref_index], reconstructed[0]).numpy()
lpips = dppp.lpips_alex(video_hr[ref_index], reconstructed[0]).numpy()
print(f"Reconstructed PSNR: {psnr:0.2f}, LPIPS: {lpips:0.4f}")

# Visualize
visualize.draw_images([video_hr[ref_index], video_lr[ref_index], reconstructed[0]])


# Using a Gaussian Blur Kernel


In [ ]:
scale_factor = 4
example = datasets.get_one_example(dataset, index=2)
video_hr = example["hr"]

kernel = dppp.conv2D_filter_rgb(dppp.ZHANG_GAUSSIAN_KERNELS[1])
video_lr = dppp.create_convolve_resize_fn(kernel, mode="wrap")(
    video_hr, scale_factor, False
)
ref_index = tf.shape(video_hr)[0] // 2


### Reconstruct


In [ ]:
# Load the denoiser
denoiser, (denoiser_min, denoiser_max) = dppp.load_denoiser("models/drugan+_0.0-0.2.h5")
num_frames = 9

# Crop the video
video_frames = video_lr.shape[0]
if video_frames > num_frames:
    start_index = max(0, ref_index - (num_frames // 2))
    end_index = start_index + num_frames
    video_lr_cropped = video_lr[start_index:end_index]
    ref_index_cropped = ref_index - start_index
else:
    video_lr_cropped = video_lr
    ref_index_cropped = ref_index

# Callbacks: Print the PSNR every 5th step
callbacks = [dppp.callback_print_psnr("psnr", 5, video_hr[ref_index])]

reconstructed = dppp.hqs_video_super_resolve(
    degraded=video_lr_cropped,
    ref_index=ref_index_cropped,
    sr_factor=scale_factor,
    denoiser=denoiser,
    max_denoiser_stddev=denoiser_max,
    kernel=kernel,
    callbacks=callbacks,
)


In [ ]:
# Print PSNR and LPIPS
psnr = dppp.psnr(video_hr[ref_index], reconstructed[0]).numpy()
lpips = dppp.lpips_alex(video_hr[ref_index], reconstructed[0]).numpy()
print(f"Reconstructed PSNR: {psnr:0.2f}, LPIPS: {lpips:0.4f}")

# Visualize
visualize.draw_images([video_hr[ref_index], video_lr[ref_index], reconstructed[0]])


# Unkown Downscaling

In [ ]:
scale_factor = 4
resize_fn = dppp.create_resize_fn("bicubic", True)

dataset = tfds.load("mdsp_color_sr", split="test").map(
    datasets.compose(
        datasets.get_value("video"),
        datasets.to_float32,
        datasets.from_255_to_1_range,
    )
)

video_lr = datasets.get_one_example(dataset, index=0)
ref_index = tf.shape(video_lr)[0] // 2

### Reconstruct


In [ ]:
# Load the denoiser
denoiser, (denoiser_min, denoiser_max) = dppp.load_denoiser("models/drugan+_0.0-0.2.h5")
num_frames = 9
resize_fn = dppp.create_resize_fn(resize_method, True)

# Crop the video
video_frames = video_lr.shape[0]
if video_frames > num_frames:
    start_index = max(0, ref_index - (num_frames // 2))
    end_index = start_index + num_frames
    video_lr_cropped = video_lr[start_index:end_index]
    ref_index_cropped = ref_index - start_index
else:
    video_lr_cropped = video_lr
    ref_index_cropped = ref_index

reconstructed = dppp.hqs_video_super_resolve(
    degraded=video_lr_cropped,
    ref_index=ref_index_cropped,
    sr_factor=scale_factor,
    denoiser=denoiser,
    max_denoiser_stddev=denoiser_max,
    resize_fn=resize_fn,
)


In [ ]:
visualize.draw_images([video_lr[ref_index], reconstructed[0]])